__Context__: We are simulating a scenario where our company, Netfluxter (Netfluxter allows customer to rent Netflix movies), has data stored in different locations like a MySQL server, MongoDB, and locally as CSV files. The goal is to gather information from these different sources and build a dimensional data mart that mirrors a specific business process, such as retail sales, inventory management, etc.

Specifically, for Netfluxter, we'll be using data from the Sakila MySQL database for customer-related datails like payments and rentals. Payment data will be uploaded onto MongoDB before we start the ETL process, mimicking a real scenario. For movie data, we'll use the "Netflix Movies and TV Shows" dataset available on Kaggle. My task is to design and create a dimensional data mart focused on rental transactions. The fact table (fact_rentals) will allow Netfluxter to analyze various aspect of their rental operations, such as rental trends over time, popular movies, store performance, etc.

__Task__: First, design the dimensional data mart. Then, develop an ETL pipeline to extract, transform, and load data into the data mart. Finally, write SQL queries to demonstrate that everything is functioning correctly.

__Step 0: Setting Up__

Placing data in their destinated locations to simulate the scenario that Netfluxter has different data stored in various different location.

Data and their locations:
- netflix_title.csv (local on pc)
- sakila_payment database (local MongoDB)
- sakila database (MySQL server)

In [68]:
# import necessary libraries
import os
import json
import numpy as np
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [69]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.29
Running PyMongo Version: 4.6.3


In [70]:
# declare and assign connection variables
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "My0228middleSQL"

src_dbname = "sakila"
dst_dbname = "netfluxter_dw"

# declare and assign connection variables for the MongoDB Server
mongodb_args = {
    "user_name" : "",
    "password" : "password",
    "cluster_name" : "cluster_name",
    "cluster_subnet" : "xxxxx",
    "cluster_location" : "local", # "atlas"
    "db_name" : "sakila_payment"
}

In [71]:
# define functions for getting data from and setting data into databases
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()

    return dframe

def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)

    if db_operation == "insert":
        df.to_sql(table_name, con=sqlEngine, index=False, if_exists="replace")
        with sqlEngine.connect() as connection:
            connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))

    elif db_operation == "update":
        df.to_sql(table_name, con=sqlEngine, index=False, if_exists="append")

    connection.close()

def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [72]:
# populate MongoDB with source data
client = get_mongo_client(**mongodb_args)

# gets the path of the Current Working Directory for this Notebook, and then Appends the 'data' directory
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"payment" : 'sakila_payment.json'}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)    

In [73]:
# create the new data warehouse database (netfluxter_dw), and to use it, switch the connection context
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

# drop a database if it already exists and create a new one
with sqlEngine.connect() as connection:
    connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
    connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
    connection.execute(text(f"USE {dst_dbname};"))

In [74]:
# create the dim_date table in the netfluxter_dw by executing the lab2c create_populate_dim_date SQL script


__Step 1: Create and Populate Dimension Tables__

1.1 Extract data from the source database tables to build dim tables

In [84]:
# from MySQL
# customer
sql_customer = "SELECT * FROM sakila.customer;"
df_customer = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customer)
df_customer.head(2)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [87]:
# from MySQL
# store
sql_store = "SELECT * FROM sakila.store;"
df_store = get_dataframe(user_id, pwd, host_name, src_dbname, sql_store)
df_store.head(2)

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2006-02-15 04:57:12
1,2,2,2,2006-02-15 04:57:12


In [77]:
# from MySQL
# address
sql_address = "SELECT * FROM sakila.address;"
df_address = get_dataframe(user_id, pwd, host_name, src_dbname, sql_address)
df_address.head(2)

,address_id,address,address2,district,city_id,postal_code,phone,location,last_update
0,1,47 MySakila Drive,None,Alberta,300,,,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00>\n2]c4\...,2014-09-25 22:30:27
1,2,28 MySQL Boulevard,None,QLD,576,,,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x8e\x10...,2014-09-25 22:30:09


In [78]:
# from MySQL
# city
sql_city = "SELECT * FROM sakila.city;"
df_city = get_dataframe(user_id, pwd, host_name, src_dbname, sql_city)
df_city.head(2)

,city_id,city,country_id,last_update
0,1,A Coruña (La Coruña),87,2006-02-15 04:45:25
1,2,Abha,82,2006-02-15 04:45:25


In [79]:
# from MySQL
# country
sql_country = "SELECT * FROM sakila.country;"
df_country = get_dataframe(user_id, pwd, host_name, src_dbname, sql_country)
df_country.head(2)

,country_id,country,last_update
0,1,Afghanistan,2006-02-15 04:44:00
1,2,Algeria,2006-02-15 04:44:00


In [80]:
# from MySQL
# dim_date
sql_dim_date = "SELECT date_key, full_date FROM netfluxter_dw.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_date)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


In [81]:
# from local CSV file
# movie
data_dir = os.path.join(os.getcwd(), 'data')
data_file = os.path.join(data_dir, 'netflix_titles.csv')
df_movie = pd.read_csv(data_file, header=0, index_col=0).reset_index()

df_movie.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


1.2 Perform any necessary transformation

In [85]:
# customer
# create a list that lists the names of each column that will be removed
drop_cols = ['address_id', 'create_date', 'last_update']
df_customer.drop(drop_cols, axis=1, inplace=True)

# replace values in the "active" column (1 -> active, 0 -> inactive)
df_customer['active'] = df_customer['active'].replace({1: 'active', 0: 'inactive'})

# insert a new column to serve as the primary key
df_customer.insert(0, "customer_key", range(1, df_customer.shape[0]+1))

# reorder columns
ordered_columns = ['customer_key', 'customer_id', 'first_name', 
                   'last_name', 'email', 'store_id',
                   'active']
df_customer = df_customer[ordered_columns]

df_customer.head(2)

,customer_key,customer_id,first_name,last_name,email,store_id,active
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,1,active
1,2,2,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,1,active


In [88]:
# store
drop_cols = ['manager_staff_id', 'last_update']
df_store.drop(drop_cols, axis=1, inplace=True)

'''NOTE: 
We want to transform the store table to contain the address of the store instead of a foreign key
('address_id') to the address table, but the address table contains a foreign key (city_id) to the city table
and the city table contains a foreign key (country_id) to the country table. We will perform
the merge operations more than once to achieve what we want.
'''

# join df_store with df_address
df_store_address = pd.merge(df_store, df_address, on='address_id', how='left')

# join df_store_address with df_city
df_store_address_city = pd.merge(df_store_address, df_city, on='city_id', how='left')

# join df_store_address_city with df_country
df_store_address_city_country = pd.merge(df_store_address_city, df_country, on='country_id', how='left')

# rename df_store_address_city_country to df_store for simplicity
df_store = df_store_address_city_country
                            
# drop redundant/unwanted columns
drop_cols = ['address_id', 'city_id', 'phone', 'location', 'last_update_x', 'country_id', 'last_update_y', 'last_update']
df_store.drop(drop_cols, axis=1, inplace=True)

df_store.insert(0, "store_key", range(1, df_store.shape[0]+1))

ordered_columns = ['store_key', 'store_id', 'address', 
                   'address2', 'city', 'district',
                   'postal_code', 'country']
df_store = df_store[ordered_columns]

df_store.head(2)

,store_key,store_id,address,address2,city,district,postal_code,country
0,1,1,47 MySakila Drive,None,Lethbridge,Alberta,,Canada
1,2,2,28 MySQL Boulevard,None,Woodridge,QLD,,Australia


In [89]:
# movie
'''NOTE:
The Sakila database includes customer information between 
May 24, 2005, and Feb. 14, 2006. In contrast, the Netflix 
movies and TV shows database contains movies released after 
2006. Therefore, Sakila's customers might be renting movies 
released in the future. However, for this project, we'll set 
aside realism.
'''
# removing all TV shows, leaving only movies
df_movie = df_movie[df_movie['type'] != 'TV Show']

# remove unwanted columns
drop_cols = ['type', 'date_added']
df_movie.drop(drop_cols, axis=1, inplace=True)

# rename 'show_id' to 'movie_id'
df_movie.rename(columns={'show_id': 'movie_id'}, inplace=True)

df_movie.insert(0, "movie_key", range(1, df_movie.shape[0]+1))

df_movie.head(2)

,movie_key,movie_id,title,director,cast,country,release_year,rating,duration,listed_in,description
0,1,s1,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
6,2,s7,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...


1.3 Load the transformed dataframes into netfluxter_dw on MySQL server by creating new tables

In [90]:
db_operation = "insert"

tables = [('dim_customers', df_customer, 'customer_key'),
          ('dim_stores', df_store, 'store_key'),
          ('dim_movies', df_movie, 'movie_key')]

In [91]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

1.4 Download the dim_movies table as a CSV file and store it on local pc (because Netfluxter exec insisted I guess...)

In [92]:
data_dir = os.path.join(os.getcwd(), 'data')
dest_file = os.path.join(data_dir, 'dim_movies.csv')

df_movie.to_csv(dest_file)

__Step 2: Create and Populate the Fact Table__

2.1 Extract necessary data from all the different sources

In [93]:
# from MongoDB
# payment
client = get_mongo_client(**mongodb_args)

query = {} # select all elements (columns), and all documents (rows).
collection = "payment"

df_payment = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)

# removing unnecessary columns not needed for the fact_rentals table
drop_cols = ['customer_id', 'staff_id', 'payment_date', 'last_update']
df_payment.drop(drop_cols, axis=1, inplace=True)

# rename 'amount' to 'payment_amount'
df_payment.rename(columns={'amount': 'payment_amount'}, inplace=True)

df_payment.head(2)

,payment_id,rental_id,payment_amount
0,1,76,2.99
1,2,573,0.99


In [94]:
# from MySQL
# rental
sql_rental = "SELECT * FROM sakila.rental;"
df_fact_rentals = get_dataframe(user_id, pwd, host_name, src_dbname, sql_rental)
df_fact_rentals.head(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


In [95]:
# from MySQL
# dim_customers
sql_dim_customer = "SELECT * FROM netfluxter_dw.dim_customers;"
df_dim_customers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_customer)
df_dim_customers.head(2)

,customer_key,customer_id,first_name,last_name,email,store_id,active
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,1,active
1,2,2,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,1,active


In [96]:
# from MySQL
# dim_stores
sql_dim_stores = "SELECT * FROM netfluxter_dw.dim_stores;"
df_dim_stores = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_stores)
df_dim_stores.head(2)

,store_key,store_id,address,address2,city,district,postal_code,country
0,1,1,47 MySakila Drive,None,Lethbridge,Alberta,,Canada
1,2,2,28 MySQL Boulevard,None,Woodridge,QLD,,Australia


In [97]:
# local csv file
# dim_movies
data_dir = os.path.join(os.getcwd(), 'data')
data_file = os.path.join(data_dir, 'dim_movies.csv')
df_dim_movies = pd.read_csv(data_file, header=0, index_col=0)
df_dim_movies.head(2)

,movie_key,movie_id,title,director,cast,country,release_year,rating,duration,listed_in,description
0,1,s1,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
6,2,s7,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...


2.2 Construct the fact_rentals table

In [98]:
# start by dropping unwanted columns in df_rental
drop_cols = ['inventory_id', 'staff_id', 'last_update']
df_fact_rentals.drop(drop_cols, axis=1, inplace=True)

df_fact_rentals.head(2)

,rental_id,rental_date,customer_id,return_date
0,1,2005-05-24 22:53:30,130,2005-05-26 22:04:30
1,2,2005-05-24 22:54:33,459,2005-05-28 19:40:33


In [99]:
# add 'movie.id' as a column, 'movie_id' is a foreign key to the dim_movies table
''' NOTE: 
In the sakila dataset, the 'rental' table includes an 'inventory_id' 
column, which is a foreign key linking to a movie in the 'inventory' 
table. This connection helps identify which movie is associated 
with each rental transaction through the 'rental_id' in the 
'rental' table. However, since we're "renting out" Netflix movies 
instead of movies from the sakila inventory, the original link 
between 'inventory_id' and 'rental_id' no longer applies. Thus,
I'll just populate the 'movie_id' column in the 'fact_rentals' 
table with randomly selected movies from the Netflix dataset. 
This way, each rental transaction will be linked to a movie in 
the Netflix dataset.
'''
# add a new column 'movie_id' to df_rental
df_fact_rentals['movie_id'] = ""

# populate 'movie_id' column in df_rental with random values from df_movie['movie_id']
random_values = np.random.choice(df_dim_movies['movie_id'], len(df_fact_rentals))
df_fact_rentals['movie_id'] = random_values

df_fact_rentals.head(2)

,rental_id,rental_date,customer_id,return_date,movie_id
0,1,2005-05-24 22:53:30,130,2005-05-26 22:04:30,s5319
1,2,2005-05-24 22:54:33,459,2005-05-28 19:40:33,s4890


In [100]:
# add 'store_id' as a column, 'store_id' is a foreign key to the dim_stores table
# merge df_rental and df_customer on customer_id so df_dental can have a populated 'store_id' column based on 'customer_id'
df_fact_rentals = pd.merge(df_fact_rentals, df_dim_customers, on='customer_id', how='left')

# drop unwanted columns
drop_cols = ['customer_key', 'first_name', 'last_name', 'email', 'active']
df_fact_rentals.drop(drop_cols, axis=1, inplace=True)

df_fact_rentals.head(2)

,rental_id,rental_date,customer_id,return_date,movie_id,store_id
0,1,2005-05-24 22:53:30,130,2005-05-26 22:04:30,s5319,1
1,2,2005-05-24 22:54:33,459,2005-05-28 19:40:33,s4890,1


In [101]:
# add 'rental_duration' as a column and populate it
# convert rental_date and return_date to datetime objects
df_fact_rentals['rental_date'] = pd.to_datetime(df_fact_rentals['rental_date'])
df_fact_rentals['return_date'] = pd.to_datetime(df_fact_rentals['return_date'])

# calculate rental_duration in days
df_fact_rentals['rental_duration'] = (df_fact_rentals['return_date'] - df_fact_rentals['rental_date']).dt.days

df_fact_rentals.head(2)

,rental_id,rental_date,customer_id,return_date,movie_id,store_id,rental_duration
0,1,2005-05-24 22:53:30,130,2005-05-26 22:04:30,s5319,1,1.0
1,2,2005-05-24 22:54:33,459,2005-05-28 19:40:33,s4890,1,3.0


In [102]:
# replace 'rental_date' with 'rental_date_key', and replace 'return_date' with 'return_date_key' that points to the same date in the dim_date table
# fetch the primary key (date_key) and the business key (full_date) from the dim_date table
sql_dim_date = "SELECT date_key, full_date FROM netfluxter_dw.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_date)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

# for each date typed column (rental_date, return_date) in fact_rentals, replace that column with primary key (date_key) that correspond with that column rental date
# rental date
df_dim_rental_date = df_dim_date.rename(columns={"date_key" : "rental_date_key", "full_date" : "rental_date"})
df_fact_rentals.rental_date = df_fact_rentals.rental_date.astype('datetime64[ns]').dt.date

df_fact_rentals = pd.merge(df_fact_rentals, df_dim_rental_date, on='rental_date', how='left')
df_fact_rentals.drop(['rental_date'], axis=1, inplace=True)

# return date
df_dim_return_date = df_dim_date.rename(columns={"date_key" : "return_date_key", "full_date" : "return_date"})
df_fact_rentals.return_date = df_fact_rentals.return_date.astype('datetime64[ns]').dt.date

df_fact_rentals = pd.merge(df_fact_rentals, df_dim_return_date, on='return_date', how='left')
df_fact_rentals.drop(['return_date'], axis=1, inplace=True)

df_fact_rentals.head(2)

,rental_id,customer_id,movie_id,store_id,rental_duration,rental_date_key,return_date_key
0,1,130,s5319,1,1.0,20050524,20050526.0
1,2,459,s4890,1,3.0,20050524,20050528.0


In [103]:
# add 'payment' as a column, 'payment' contains the amount paid associated with each unique rental transaction
# merge df_fact_rentals and df_payment base on rental_id so df_rental can have a populated 'payment_amount' column based on 'rental_id'
df_fact_rentals = pd.merge(df_fact_rentals, df_payment, on='rental_id', how='inner')

# drop unwanted columns
drop_cols = ['payment_id']
df_fact_rentals.drop(drop_cols, axis=1, inplace=True)

df_fact_rentals.head(2)

,rental_id,customer_id,movie_id,store_id,rental_duration,rental_date_key,return_date_key,payment_amount
0,18,19,s3640,1,6.0,20050525,20050531.0,0.99
1,46,7,s6831,1,8.0,20050525,20050602.0,5.99


In [104]:
# reorder columns
ordered_columns = ['rental_id', 'customer_id', 'store_id', 
                   'movie_id', 'payment_amount', 'rental_duration',
                   'rental_date_key', 'return_date_key']
df_fact_rentals = df_fact_rentals[ordered_columns]

df_fact_rentals.head(2)

,rental_id,customer_id,store_id,movie_id,payment_amount,rental_duration,rental_date_key,return_date_key
0,18,19,1,s3640,0.99,6.0,20050525,20050531.0
1,46,7,1,s6831,5.99,8.0,20050525,20050602.0


In [105]:
# insert a new column, with an ever-incrementing numeric value, to serve as the primary key
df_fact_rentals.insert(0, 'fact_rental_key', range(1, df_fact_rentals.shape[0]+1))

df_fact_rentals.head(2)

,fact_rental_key,rental_id,customer_id,store_id,movie_id,payment_amount,rental_duration,rental_date_key,return_date_key
0,1,18,19,1,s3640,0.99,6.0,20050525,20050531.0
1,2,46,7,1,s6831,5.99,8.0,20050525,20050602.0


2.3 Write the fact_rentals data frame back to netfluxter_dw database

In [106]:
table_name = "fact_rentals"
primary_key = "fact_rental_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_rentals, table_name, primary_key, db_operation)

Step 3. Demonstrate that the netfluxter data warehouse exists and contains the correct data

In [108]:
# SQL script that selects:
## movies that are have been rented more than once
## names of the customers who rented those movies
## amount each customer paid to rent those movies

sql_test = """
    SELECT dm.title AS movie_title,
        dc.first_name AS customer_first_name,
        dc.last_name AS customer_last_name,
        fr.payment_amount AS rental_payment_amount
    FROM netfluxter_dw.fact_rentals fr
    JOIN netfluxter_dw.dim_movies dm ON fr.movie_id = dm.movie_id
    JOIN netfluxter_dw.dim_customers dc ON fr.customer_id = dc.customer_id
    WHERE fr.movie_id IN (
        SELECT movie_id
        FROM fact_rentals
        GROUP BY movie_id
        HAVING COUNT(*) > 1
);
""".format(dst_dbname)

df_test = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_test)
df_test.head()

,movie_title,customer_first_name,customer_last_name,rental_payment_amount
0,Fantastic Fungi,RUTH,MARTINEZ,2.99
1,Fantastic Fungi,LINDA,WILLIAMS,5.99
2,9to5: The Story of a Movement,DOROTHY,TAYLOR,2.99
3,9to5: The Story of a Movement,DEBORAH,WALKER,2.99
4,Chhota Bheem & Krishna: Pataliputra- City of t...,DEBORAH,WALKER,0.99
